In [5]:
import pandas as pd
import quandl
quandl.ApiConfig.api_key = "aiiKxAtb7B1s2h5FLLve"

In [2]:
import sys
sys.executable

'/anaconda3/envs/edgar/bin/python'

In [14]:
mydata = quandl.get("WIKI/AAPL")

In [19]:
end_date = "2019-01-01"
mydata = quandl.get("WIKI/GOOGL", end_date=end_date)

In [20]:
mydata.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0


In [23]:
from datetime import datetime
from iexfinance.stocks import get_historical_data

start = datetime(2017, 1, 1)
end = datetime(2018, 1, 1)

df = get_historical_data("TSLA", start, end, output_format='pandas')

In [26]:
df.head()

,open,high,low,close,volume
date,,,,,
2017-01-03,214.86,220.33,210.96,216.99,5923254
2017-01-04,214.75,228.00,214.31,226.99,11213471
2017-01-05,226.42,227.48,221.95,226.75,5911695
2017-01-06,226.93,230.31,225.45,229.01,5527893
2017-01-09,228.97,231.92,228.00,231.28,3979484


In [29]:
import os, json
import pandas as pd

import utils


nums_folder = "./data/sec"
num_prefix = "numbers_sec-numbers"
df_nums = utils.load_folder_csv(nums_folder, prefix=num_prefix)
tags = json.loads(open('freq_tags.json', 'r').read())
df_quarters = df_nums[(df_nums['number_of_quarters']==1) | (df_nums['number_of_quarters']==4)
                     | (df_nums['number_of_quarters']==0)]
df_quarters_tag = df_quarters[df_quarters['measure_tag'].isin(tags)]

/anaconda3/envs/edgar/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [30]:
len(df_quarters_tag)

2845832

In [31]:
df_quarters_tag.head()

,submission_number,company_name,measure_tag,version,period_end_date,number_of_quarters,units,dimension_hash,iprx,value,footnote,footnote_length,num_dimensions,coregistrant,duration_fraction,date_fraction,decimal
9,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0x500a45ef2d01a667a6328c74ea5c5c2a,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3
32,0001609711-15-000009,GODADDY INC.,GeneralAndAdministrativeExpense,us-gaap/2015,20150430,0,USD,0xadbf96d8c8068c9b134f2ba2fb344644,0,26600000.0,NaN,0,5,NaN,-0.317808,0.0,-5
44,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0xb3eb582260e0d15fe7138daf0d9b4ced,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3
51,0001564590-15-003812,INTERNATIONAL STEM CELL CORP,StockIssuedDuringPeriodSharesNewIssues,us-gaap/2014,20140930,0,shares,0x74a0e1a829411288d1a8703d538432fa,0,500.0,NaN,0,5,NaN,-0.010958,-14.0,32767
62,0000078003-15-000026,PFIZER INC,DerivativeInstrumentsGainLossReclassifiedFromA...,us-gaap/2015,20150331,1,USD,0xc31fe30aa8d64c36a318796ba02cb81e,0,373000000.0,NaN,0,5,NaN,0.046576,2.0,-6


In [35]:
import re, requests

def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}
    for ticker in TICKERS:
        f = requests.get(URL.format(ticker), stream = True)
        results = CIK_RE.findall(f.text)
        if len(results):
            results[0] = int(re.sub('\.[0]*', '.', results[0]))
            cik_dict[str(ticker).upper()] = str(results[0])
            
    with open('cik.json', "w") as f:
        f.write(json.dumps(cik_dict))
    
    print(cik_dict)

getCIKs(['wmt','amzn','nflx'])
# returns:
# {'WMT': '104169', 'AMZN': '1018724', 'NFLX': '1065280'}

{'WMT': '104169', 'AMZN': '1018724', 'NFLX': '1065280'}


In [40]:
# From http://rankandfiled.com/#/data/tickers
cik = pd.read_csv("cik_ticker.csv", sep='|')
cik.head()

,CIK,Ticker,Name,Exchange,SIC,Business,Incorporated,IRS
0,1090872,A,Agilent Technologies Inc,NYSE,3825.0,CA,DE,770518772.0
1,4281,AA,Alcoa Inc,NYSE,3350.0,PA,PA,250317820.0
2,1332552,AAACU,Asia Automotive Acquisition Corp,NaN,6770.0,DE,DE,203022522.0
3,1287145,AABB,Asia Broadband Inc,OTC,8200.0,GA,NV,721569126.0
4,1024015,AABC,Access Anytime Bancorp Inc,NaN,6035.0,NM,DE,850444597.0


In [60]:
df_quarters_tag['cik'] = df_quarters_tag['submission_number'].str.split('-', expand=True)[0]

/anaconda3/envs/edgar/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
df_quarters_tag.head()

,submission_number,company_name,measure_tag,version,period_end_date,number_of_quarters,units,dimension_hash,iprx,value,footnote,footnote_length,num_dimensions,coregistrant,duration_fraction,date_fraction,decimal,cik
9,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0x500a45ef2d01a667a6328c74ea5c5c2a,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3,0001193125
32,0001609711-15-000009,GODADDY INC.,GeneralAndAdministrativeExpense,us-gaap/2015,20150430,0,USD,0xadbf96d8c8068c9b134f2ba2fb344644,0,26600000.0,NaN,0,5,NaN,-0.317808,0.0,-5,0001609711
44,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0xb3eb582260e0d15fe7138daf0d9b4ced,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3,0001193125
51,0001564590-15-003812,INTERNATIONAL STEM CELL CORP,StockIssuedDuringPeriodSharesNewIssues,us-gaap/2014,20140930,0,shares,0x74a0e1a829411288d1a8703d538432fa,0,500.0,NaN,0,5,NaN,-0.010958,-14.0,32767,0001564590
62,0000078003-15-000026,PFIZER INC,DerivativeInstrumentsGainLossReclassifiedFromA...,us-gaap/2015,20150331,1,USD,0xc31fe30aa8d64c36a318796ba02cb81e,0,373000000.0,NaN,0,5,NaN,0.046576,2.0,-6,0000078003


In [62]:
cik['cik'] = cik['CIK'].astype(str).str.zfill(10)
cik.head()

,CIK,Ticker,Name,Exchange,SIC,Business,Incorporated,IRS,cik
0,1090872,A,Agilent Technologies Inc,NYSE,3825.0,CA,DE,770518772.0,0001090872
1,4281,AA,Alcoa Inc,NYSE,3350.0,PA,PA,250317820.0,0000004281
2,1332552,AAACU,Asia Automotive Acquisition Corp,NaN,6770.0,DE,DE,203022522.0,0001332552
3,1287145,AABB,Asia Broadband Inc,OTC,8200.0,GA,NV,721569126.0,0001287145
4,1024015,AABC,Access Anytime Bancorp Inc,NaN,6035.0,NM,DE,850444597.0,0001024015


In [74]:
len(cik)

13737

In [64]:
df_merged = df_quarters_tag.merge(cik[['cik', 'Ticker']], how='left', on='cik')

In [65]:
df_merged.count()

submission_number     2845832
company_name          2845832
measure_tag           2845832
version               2845832
period_end_date       2845832
number_of_quarters    2845832
units                 2845832
dimension_hash        2845832
iprx                  2845832
value                 2777859
footnote                 8108
footnote_length       2845832
num_dimensions        2845832
coregistrant           211360
duration_fraction     2845832
date_fraction         2845832
decimal               2845832
cik                   2845832
Ticker                 837998
dtype: int64

In [72]:
len(set(df_merged['company_name'].values))

10676

In [73]:
len(set(df_merged['Ticker'].values))

2183

In [75]:
df_merged.head()

,submission_number,company_name,measure_tag,version,period_end_date,number_of_quarters,units,dimension_hash,iprx,value,footnote,footnote_length,num_dimensions,coregistrant,duration_fraction,date_fraction,decimal,cik,Ticker
0,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0x500a45ef2d01a667a6328c74ea5c5c2a,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3,0001193125,NaN
1,0001609711-15-000009,GODADDY INC.,GeneralAndAdministrativeExpense,us-gaap/2015,20150430,0,USD,0xadbf96d8c8068c9b134f2ba2fb344644,0,26600000.0,NaN,0,5,NaN,-0.317808,0.0,-5,0001609711,NaN
2,0001193125-15-188313,POLYMER GROUP INC,DerivativeGainLossOnDerivativeNet,us-gaap/2013,20141231,4,USD,0xb3eb582260e0d15fe7138daf0d9b4ced,0,0.0,NaN,0,6,NaN,-0.021917,0.0,-3,0001193125,NaN
3,0001564590-15-003812,INTERNATIONAL STEM CELL CORP,StockIssuedDuringPeriodSharesNewIssues,us-gaap/2014,20140930,0,shares,0x74a0e1a829411288d1a8703d538432fa,0,500.0,NaN,0,5,NaN,-0.010958,-14.0,32767,0001564590,NaN
4,0000078003-15-000026,PFIZER INC,DerivativeInstrumentsGainLossReclassifiedFromA...,us-gaap/2015,20150331,1,USD,0xc31fe30aa8d64c36a318796ba02cb81e,0,373000000.0,NaN,0,5,NaN,0.046576,2.0,-6,0000078003,PFE


In [90]:
cik[cik['Name'].str.startswith('God')]

,CIK,Ticker,Name,Exchange,SIC,Business,Incorporated,IRS,cik


In [78]:
cik['Name']

0                              Agilent Technologies Inc
1                                             Alcoa Inc
2                      Asia Automotive Acquisition Corp
3                                    Asia Broadband Inc
4                            Access Anytime Bancorp Inc
5        Sinocoking Coal & Coke Chemical Industries Inc
6                         Asset Acceptance Capital Corp
7                                  Ace Cash Express Inc
8                                All American Gold Corp
9                                  Airtran Holdings Inc
10                                        Aaipharma Inc
11                                         Avantair Inc
12                          American Airlines Group Inc
13                       American Asset Management Corp
14                     Altisource Asset Management Corp
15                               Atlantic American Corp
16                                      Adera Mines LTD
17                              American Ammunit